# Evolver Loop 3 - LB Feedback Analysis

**Submission Result:** CV 70.6827 | LB 70.6827 (gap: +0.0000)

Perfect CV-LB alignment! This is an optimization problem, not a prediction problem, so there's no distribution shift.

**Target:** 68.922808
**Gap:** 1.76 points (2.5%)

In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Check what pre-optimized CSVs we have
preopt_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized'
print('Available pre-optimized CSVs:')
for root, dirs, files in os.walk(preopt_dir):
    for f in files:
        if f.endswith('.csv'):
            path = os.path.join(root, f)
            size = os.path.getsize(path) / 1024
            print(f'  {path}: {size:.1f} KB')

Available pre-optimized CSVs:
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv: 1415.5 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/submission.csv: 1360.2 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv: 1558.3 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv: 1558.3 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv: 1310.0 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv: 1730.6 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted/72.49.csv: 1310.0 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted/71.97.csv: 1730.6 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv: 1558.3 KB
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimi

In [2]:
# Load our current best submission and analyze per-N scores
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 30
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_score(trees, n):
    if not trees:
        return 0.0
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    side = Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor
    return float(side ** 2 / Decimal(n))

def load_configuration_from_df(n, df):
    group_data = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row['x'])[1:] if str(row['x']).startswith('s') else str(row['x'])
        y = str(row['y'])[1:] if str(row['y']).startswith('s') else str(row['y'])
        deg = str(row['deg'])[1:] if str(row['deg']).startswith('s') else str(row['deg'])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print('Helper functions defined')

Helper functions defined


In [3]:
# Analyze per-N scores for our current best submission
df = pd.read_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv')

per_n_scores = []
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if trees:
        score = get_score(trees, n)
        per_n_scores.append({'n': n, 'score': score})

per_n_df = pd.DataFrame(per_n_scores)
print(f'Total score: {per_n_df["score"].sum():.6f}')
print(f'\nTop 10 worst-performing N values (highest scores):')
print(per_n_df.nlargest(10, 'score'))

Total score: 70.682741

Top 10 worst-performing N values (highest scores):
     n     score
0    1  0.661250
1    2  0.450779
2    3  0.434745
4    5  0.416850
3    4  0.416545
6    7  0.399897
5    6  0.399610
8    9  0.387415
7    8  0.385407
14  15  0.379203


In [4]:
# Compare with target - what improvement do we need per N?
target_total = 68.922808
current_total = per_n_df['score'].sum()
gap = current_total - target_total

print(f'Current total: {current_total:.6f}')
print(f'Target total: {target_total:.6f}')
print(f'Gap: {gap:.6f} ({gap/current_total*100:.2f}%)')
print(f'\nAverage improvement needed per N: {gap/200:.6f}')
print(f'\nIf we improve the worst 20 N values by 10% each:')
worst_20 = per_n_df.nlargest(20, 'score')
worst_20_total = worst_20['score'].sum()
print(f'  Worst 20 total: {worst_20_total:.6f}')
print(f'  10% improvement: {worst_20_total * 0.10:.6f}')
print(f'  Would close gap? {worst_20_total * 0.10 > gap}')

Current total: 70.682741
Target total: 68.922808
Gap: 1.759933 (2.49%)

Average improvement needed per N: 0.008800

If we improve the worst 20 N values by 10% each:
  Worst 20 total: 8.077149
  10% improvement: 0.807715
  Would close gap? False


In [5]:
# Check if jonathanchan C++ SA optimizer is available
import subprocess

# Check if we can compile the C++ optimizer
print('Checking C++ compiler...')
result = subprocess.run(['g++', '--version'], capture_output=True, text=True)
print(result.stdout[:100])

Checking C++ compiler...
g++ (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is 


In [6]:
# Summary of findings
print('='*60)
print('SUMMARY')
print('='*60)
print(f'\nCurrent best: 70.6827 (LB verified)')
print(f'Target: 68.9228')
print(f'Gap: 1.76 points (2.5%)')
print(f'\nKey observations:')
print('1. CV = LB (perfect alignment - this is optimization, not prediction)')
print('2. Pre-optimized baseline is at local optimum for bbox3')
print('3. Backward propagation found tiny improvement (0.0017)')
print('4. Ensemble of all CSVs only improved by 0.003')
print(f'\nNext steps to try:')
print('1. Compile jonathanchan C++ SA optimizer with fractional translation')
print('2. Run multi-generation optimization (opt_v3 with population)')
print('3. Try BackPacking approach (backward iteration from N=200)')
print('4. Focus on worst-performing N values')

SUMMARY

Current best: 70.6827 (LB verified)
Target: 68.9228
Gap: 1.76 points (2.5%)

Key observations:
1. CV = LB (perfect alignment - this is optimization, not prediction)
2. Pre-optimized baseline is at local optimum for bbox3
3. Backward propagation found tiny improvement (0.0017)
4. Ensemble of all CSVs only improved by 0.003

Next steps to try:
1. Compile jonathanchan C++ SA optimizer with fractional translation
2. Run multi-generation optimization (opt_v3 with population)
3. Try BackPacking approach (backward iteration from N=200)
4. Focus on worst-performing N values


In [ ]:
# Check the best_ensemble.csv and other CSVs we haven't tried
from shapely.strtree import STRtree

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def score_csv(path):
    try:
        df = pd.read_csv(path)
        total_score = 0.0
        overlaps = []
        for n in range(1, 201):
            trees = load_configuration_from_df(n, df)
            if trees:
                total_score += get_score(trees, n)
                if has_overlap(trees):
                    overlaps.append(n)
        return total_score, overlaps
    except Exception as e:
        return None, str(e)

# Score all available CSVs
csvs_to_check = [
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/bucket-of-chump/submission.csv',
]

print('Scoring additional CSVs...')
for path in csvs_to_check:
    score, overlaps = score_csv(path)
    if score:
        print(f'{os.path.basename(path)}: score={score:.6f}, overlaps={len(overlaps) if isinstance(overlaps, list) else overlaps}')
    else:
        print(f'{os.path.basename(path)}: Error - {overlaps}')